In [9]:
import pandas as pd
import scipy as sc
import numpy as np
import os
import json

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.colors import colorConverter
import seaborn as sns
import glob

%load_ext autoreload
%autoreload 2
%load_ext blackcellmagic
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [12]:
from constants import guest_types
from constants import colors
from constants import systems
from constants import experimental_deltaH
from bootstrap import bootstrap
from bootstrap import thermodynamic_bootstrap
from blocking import get_block_sem
import plotting

# Grab experimental data

In [4]:
experimental_list = experimental_deltaH.split("\n")
experimental = pd.DataFrame([i.split("\t") for i in experimental_list],
            columns=["System", "Delta H", "SEM"])
experimental["Delta H"] = pd.to_numeric(experimental["Delta H"])
experimental["SEM"] = pd.to_numeric(experimental["SEM"])

In [5]:
experimental.head()

,System,Delta H,SEM
0,a-bam,-2.17,0.05
1,a-nmb,-2.57,0.06
2,a-mba,-2.68,0.07
3,a-pam,-3.28,0.02
4,a-ham,-4.19,0.02


# Grab Niel's BGBG-TIP3P data

In [6]:
def get_energies_for_enthalpy(mden):
    """
    Modeled after Niel's version.
    """

    vdw = []
    ele = []
    bnd = []
    ang = []
    dih = []
    v14 = []
    e14 = []

    with open(mden, "r") as f:
        for line in f.readlines()[10:]:
            words = line.rstrip().split()
            if words[0] == "L6":
                vdw.append(float(words[3]))
                ele.append(float(words[4]))
            elif words[0] == "L7":
                bnd.append(float(words[2]))
                ang.append(float(words[3]))
                dih.append(float(words[4]))
            elif words[0] == "L8":
                v14.append(float(words[1]))
                e14.append(float(words[2]))
    return bnd, ang, dih, v14, e14, vdw, ele


In [20]:
for system in systems:
    end_point_energies = {}
    end_point_energies[system] = {}
    for window in ["a00", "r00"]:
        end_point_energies[system][window] = {}
        mden_files = glob.glob(
            os.path.join("/home/dslochower/kirkwood-niel/projects/cds/wat6/bgbg-tip3p/", system, window) + "/mden*"
        )
        mden_files = [i for i in mden_files if "pl" not in i]

        e_bond = []
        e_ang = []
        e_dih = []
        e_v14 = []
        e_e14 = []
        e_vdw = []
        e_ele = []

        for index, file in enumerate(mden_files):
            bnd, ang, dih, v14, e14, vdw, ele = get_energies_for_enthalpy(file)
            e_bond.append(bnd)
            e_ang.append(ang)
            e_dih.append(dih)
            e_v14.append(v14)
            e_e14.append(e14)
            e_vdw.append(vdw)
            e_ele.append(ele)

        e_bond = [item for sublist in e_bond for item in sublist]
        e_ang = [item for sublist in e_ang for item in sublist]
        e_dih = [item for sublist in e_dih for item in sublist]
        e_v14 = [item for sublist in e_v14 for item in sublist]
        e_e14 = [item for sublist in e_e14 for item in sublist]
        e_vdw = [item for sublist in e_vdw for item in sublist]
        e_ele = [item for sublist in e_ele for item in sublist]

        lists = [e_bond, e_ang, e_dih, e_v14, e_e14, e_vdw, e_ele]
        e_total = [sum(x) for x in zip(*lists)]


        # Skip SEMs right now... that is the slow part.
#         end_point_energies[system][window]["total"] = [np.mean(e_total), get_block_sem(e_total)]
#         end_point_energies[system][window]["bond"] = [np.mean(e_bond), get_block_sem(e_bond)]
#         end_point_energies[system][window]["ang"]  = [np.mean(e_ang), get_block_sem(e_ang)]
#         end_point_energies[system][window]["dih"] = [np.mean(e_dih), get_block_sem(e_dih)]
#         end_point_energies[system][window]["v14"] = [np.mean(e_v14), get_block_sem(e_v14)]
#         end_point_energies[system][window]["e14"] = [np.mean(e_e14), get_block_sem(e_e14)]
#         end_point_energies[system][window]["ele"] = [np.mean(e_ele), get_block_sem(e_ele)]
#         end_point_energies[system][window]["vdw"] = [np.mean(e_vdw), get_block_sem(e_vdw)]

        end_point_energies[system][window]["total"] = [np.mean(e_total), 1]
        end_point_energies[system][window]["bond"] = [np.mean(e_bond), 1]
        end_point_energies[system][window]["ang"]  = [np.mean(e_ang), 1]
        end_point_energies[system][window]["dih"] = [np.mean(e_dih), 1]
        end_point_energies[system][window]["v14"] = [np.mean(e_v14), 1]
        end_point_energies[system][window]["e14"] = [np.mean(e_e14), 1]
        end_point_energies[system][window]["ele"] = [np.mean(e_ele), 1]
        end_point_energies[system][window]["vdw"] = [np.mean(e_vdw), 1]

In [21]:
end_point_energies_point_energies

{'b-pnt-p': {'a00': {'ang': [95.45397955704888, 1],
   'bond': [38.62544272535933, 1],
   'dih': [147.7272861780413, 1],
   'e14': [809.3768775216431, 1],
   'ele': [-26262.936436501564, 1],
   'total': [-22014.13227654164, 1],
   'v14': [35.027296287172454, 1],
   'vdw': [3122.593277690674, 1]},
  'r00': {'ang': [95.76493645642839, 1],
   'bond': [38.722951772580224, 1],
   'dih': [146.46133177258793, 1],
   'e14': [808.8530230049261, 1],
   'ele': [-26262.600121018346, 1],
   'total': [-22013.69821419658, 1],
   'v14': [34.69329062666554, 1],
   'vdw': [3124.406373188578, 1]}}}

## Plot by orientation

## Combine each orientation

In [ ]:
def combine_data(df):

    combined = pd.DataFrame()
    df["Short"] = [i[0:-2] for i in df["System"].values]

    for hg in df["Short"].unique():
        tmp = df[df["Short"] == hg]
        for _, row in tmp.iterrows():
            for p in row["System"].split("-")[2]:
                primary_fe = row["total"]
                primary_sem = row["total_sem"]
            else:
                secondary_fe = row["total"]
                secondary_sem = row["total_sem"]
        results = thermodynamic_bootstrap(
            primary_fe, primary_sem, secondary_fe, secondary_sem
        )

        print(f"{hg} P: {primary_fe:+0.4f} ± {primary_sem:+0.2f} S: {secondary_fe:+0.2f} ± {secondary_sem:+0.2f} → Boot: {results['mean']:+0.2f} ± {results['sem']:+0.2f}")
        combined = combined.append(
            {
                "System": hg,
                "Delta H": results["mean"],
                "SEM": results["sem"],
                "Type": guest_types[hg],
            },
            ignore_index=True,
        )
    return combined
